In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from carbonplan_trace.v1.glas_allometric_eq import REALM_GROUPINGS
from carbonplan_trace.v1 import load
import carbonplan_trace.v1.model as m
import pandas as pd
from carbonplan_trace.v1.landsat_preprocess import access_credentials
import numpy as np

In [3]:
access_key_id, secret_access_key = access_credentials()

### P0
* try random split [done, doesn't help val score]
* save the concatenated file [done]
* try out afrotropics with random split to replicate the hansen landsat modeling work [done, random we can achieve good performance with random splits]
* check for infs and nulls [done]

* do HPO [in progress]
    * learning rate 
    * max depth 
    * n_estimators 

* confirm findings with different realm and different val year [done with default params]
* try normalizing all features [no benefit as expected]
* normalize all bands with NIR_V [no benefit :(]
* try first split [done, using first year as validation yields a much better validation r2 score. let's use the average of first/last as our]

### P1
do cross validation based on different years 
performance stratified by biomass value 

### P2 
examine the correlation between our continuous features 

In [4]:
realms = list(REALM_GROUPINGS.keys())
realms = ['afrotropic', 'australia', 'nearctic', 'neotropic']
# realms = ['afrotropic', 'nearctic']

In [5]:
# HPO 
import itertools

def product_dict(**kwargs):
    keys = kwargs.keys()
    vals = kwargs.values()
    for instance in itertools.product(*vals):
        yield dict(zip(keys, instance))
        
param_set = {
    'learning_rate': [0.07, 0.05, 0.03],
    'max_depth': [10, 12, 14],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'subsample': [0.5, 0.7, 0.9],
    'min_child_weight': [2, 4, 6],
    'lambda': [1, 1.5, 2],
    'alpha': [0, 0.5, 1],
    'gamma': [0, 0.5, 1],  
}

groupings = [
    ['learning_rate'],
    ['max_depth'],
    ['colsample_bytree', 'subsample', 'min_child_weight'],
    ['lambda', 'alpha', 'gamma']
]

dims = [list(range(len(param_set[g[0]]))) for g in groupings]
param_set_list = []
for orders in list(itertools.product(*dims)):
    d = {}
    for o, g in zip(orders, groupings):
        for k in g:
            d[k] = param_set[k][o]
    param_set_list.append(d)

In [6]:
def get_all_prediction_result(model, df_train, df_test, df_val): 
    
    df_train['biomass_pred'] = model._predict(df_train) 
    df_test['biomass_pred'] = model._predict(df_test) 
    df_val['biomass_pred'] = model._predict(df_val) 
    
    return pd.concat(
        [
            df_train[['lat', 'lon', 'year', 'biomass_pred']], 
            df_test[['lat', 'lon', 'year', 'biomass_pred']], 
            df_val[['lat', 'lon', 'year', 'biomass_pred']], 
        ]
    ).rename(columns={'biomass_pred': 'biomass'})


def calculate_temporal_variability(df, y1=2007, y2=2008, precision=3):
    year1 = df.loc[df.year == y1, ['lat', 'lon', 'biomass']]
    year2 = df.loc[df.year == y2, ['lat', 'lon', 'biomass']]

    year1['lat_round'] = year1.lat.round(precision)
    year1['lon_round'] = year1.lon.round(precision)
    year2['lat_round'] = year2.lat.round(precision)
    year2['lon_round'] = year2.lon.round(precision)
    
    merged = year1.merge(year2, on=['lat_round', 'lon_round'], suffixes=['_year1', '_year2'])
    
    mae = (merged.biomass_year2 - merged.biomass_year1).abs().mean()
    me = (merged.biomass_year2 - merged.biomass_year1).mean()
    
    return {'mae': mae, 'me': me}

In [7]:
scores = []
for realm in realms:
    print(f'Building model for {realm} realm')
    
    # load data, add year information
    df = load.training(realm=realm, reload=False, access_key_id=access_key_id, secret_access_key=secret_access_key)
    print(f'    size of entire df is {round(df.size / 1e9, 2)}Gb')
    
    for strategy in ['first', 'last']: 
        for random_split in [True, False]:
            # split into train/test based on year 
            df_train, df_test, df_val = m.train_test_split_based_on_year(
                df, val_strategy=strategy, random_train_test=random_split)
            print(f'    training sample size = {len(df_train)}')
            print(f'    testing sample size = {len(df_test)}')
            print(f'    eval sample size = {len(df_val)}')

            # build 2 models: 1) baseline/mean, 2) xgboost 
            # TODO: build linear model as another baseline model 
            # m.baseline_model, m.gradient_boost_model, m.random_forest_model
            for model_class in [m.xgb_model]:
                for params in [{}]:

                    model = model_class(
                        realm=realm, 
                        df_train=df_train,
                        df_test=df_test,
                        output_folder='s3://carbonplan-climatetrace/v1/models/',
                        overwrite=True,
                        validation_year=strategy,
                        params=params
                    )

                    for split, sub in zip(('train', 'test', 'val'), (df_train, df_test, df_val)):
                        model_score = model.evaluate(sub)
                        model_score['model_name'] = model.name
                        model_score['split'] = split
                        model_score['realm'] = realm
                        model_score['validation_year'] = strategy
                        model_score['random_split'] = random_split
                        model_score['sample_size'] = len(sub)
                        model_score.update(params)
                        scores.append(model_score)

scores = pd.DataFrame(scores)

Building model for afrotropic realm
    size of entire df is 0.6Gb
    training sample size = 8399642
    testing sample size = 2099911
    eval sample size = 2271212
{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'n_estimators': 999, 'random_state': 42, 'learning_rate': 0.25, 'max_depth': 14, 'colsample_bytree': 0.4, 'subsample': 0.3, 'min_child_weight': 10, 'lambda': 3, 'alpha': 2, 'gamma': 2}
    Building xgb_afrotropic_first model
[0]	validation_0-rmse:117.85918
[1]	validation_0-rmse:99.94933
[2]	validation_0-rmse:85.82634
[3]	validation_0-rmse:77.89979
[4]	validation_0-rmse:72.96287
[5]	validation_0-rmse:69.73031
[6]	validation_0-rmse:66.66892
[7]	validation_0-rmse:65.34791
[8]	validation_0-rmse:63.78269
[9]	validation_0-rmse:63.26006
[10]	validation_0-rmse:62.44613
[11]	validation_0-rmse:62.22157
[12]	validation_0-rmse:61.98031
[13]	validation_0-rmse:61.88314
[14]	validation_0-rmse:61.46103
[15]	validation_0-rmse:61.18016
[16]	validation_0-rmse:61

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


    training sample size = 7964602
    testing sample size = 2534951
    eval sample size = 2271212
{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'n_estimators': 999, 'random_state': 42, 'learning_rate': 0.25, 'max_depth': 14, 'colsample_bytree': 0.4, 'subsample': 0.3, 'min_child_weight': 10, 'lambda': 3, 'alpha': 2, 'gamma': 2}
    Building xgb_afrotropic_first model
[0]	validation_0-rmse:109.41512
[1]	validation_0-rmse:92.72817
[2]	validation_0-rmse:79.97502
[3]	validation_0-rmse:73.27895
[4]	validation_0-rmse:69.42834
[5]	validation_0-rmse:67.17607
[6]	validation_0-rmse:64.98286
[7]	validation_0-rmse:64.35215
[8]	validation_0-rmse:63.41630
[9]	validation_0-rmse:63.31747
[10]	validation_0-rmse:62.87814
[11]	validation_0-rmse:62.91461
[12]	validation_0-rmse:62.95366
[13]	validation_0-rmse:62.99468
[14]	validation_0-rmse:62.74169
[15]	validation_0-rmse:62.59959
[16]	validation_0-rmse:62.66026
[17]	validation_0-rmse:62.68107
[18]	validation_0-rmse:62.68

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


    training sample size = 8909848
    testing sample size = 2227463
    eval sample size = 1633454
{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'n_estimators': 999, 'random_state': 42, 'learning_rate': 0.25, 'max_depth': 14, 'colsample_bytree': 0.4, 'subsample': 0.3, 'min_child_weight': 10, 'lambda': 3, 'alpha': 2, 'gamma': 2}
    Building xgb_afrotropic_last model
[0]	validation_0-rmse:120.17945
[1]	validation_0-rmse:101.70274
[2]	validation_0-rmse:87.11382
[3]	validation_0-rmse:78.91506
[4]	validation_0-rmse:73.77483
[5]	validation_0-rmse:70.37069
[6]	validation_0-rmse:67.19457
[7]	validation_0-rmse:65.78554
[8]	validation_0-rmse:64.15910
[9]	validation_0-rmse:63.55584
[10]	validation_0-rmse:62.66958
[11]	validation_0-rmse:62.43179
[12]	validation_0-rmse:62.24949
[13]	validation_0-rmse:62.15939
[14]	validation_0-rmse:61.72578
[15]	validation_0-rmse:61.41154
[16]	validation_0-rmse:61.35262
[17]	validation_0-rmse:61.30171
[18]	validation_0-rmse:61.21

...

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


    training sample size = 7194278
    testing sample size = 1798570
    eval sample size = 1581558
{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'n_estimators': 999, 'random_state': 42, 'learning_rate': 0.25, 'max_depth': 14, 'colsample_bytree': 0.4, 'subsample': 0.3, 'min_child_weight': 10, 'lambda': 3, 'alpha': 2, 'gamma': 2}
    Building xgb_neotropic_last model
[0]	validation_0-rmse:152.28380
[1]	validation_0-rmse:127.91715
[2]	validation_0-rmse:108.53256
[3]	validation_0-rmse:97.48144
[4]	validation_0-rmse:90.57883
[5]	validation_0-rmse:86.22395
[6]	validation_0-rmse:81.75191
[7]	validation_0-rmse:80.05286
[8]	validation_0-rmse:77.91820
[9]	validation_0-rmse:77.22033
[10]	validation_0-rmse:76.14929
[11]	validation_0-rmse:75.84876
[12]	validation_0-rmse:75.59344
[13]	validation_0-rmse:75.48336
[14]	validation_0-rmse:74.93829
[15]	validation_0-rmse:74.59819
[16]	validation_0-rmse:74.54756
[17]	validation_0-rmse:74.50902
[18]	validation_0-rmse:74.38

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


    training sample size = 7511869
    testing sample size = 1480979
    eval sample size = 1581558
{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'tree_method': 'hist', 'n_estimators': 999, 'random_state': 42, 'learning_rate': 0.25, 'max_depth': 14, 'colsample_bytree': 0.4, 'subsample': 0.3, 'min_child_weight': 10, 'lambda': 3, 'alpha': 2, 'gamma': 2}
    Building xgb_neotropic_last model
[0]	validation_0-rmse:150.17325
[1]	validation_0-rmse:125.49618
[2]	validation_0-rmse:105.59675
[3]	validation_0-rmse:94.59754
[4]	validation_0-rmse:87.93587
[5]	validation_0-rmse:83.84218
[6]	validation_0-rmse:79.56361
[7]	validation_0-rmse:78.14428
[8]	validation_0-rmse:76.28629
[9]	validation_0-rmse:75.83379
[10]	validation_0-rmse:74.96725
[11]	validation_0-rmse:74.87154
[12]	validation_0-rmse:74.75724
[13]	validation_0-rmse:74.75967
[14]	validation_0-rmse:74.31986
[15]	validation_0-rmse:74.07568
[16]	validation_0-rmse:74.09451
[17]	validation_0-rmse:74.05637
[18]	validation_0-rmse:74.07

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [8]:
# check weighted average result 
#             'learning_rate': 0.25,
#             'max_depth': 14,
#             'colsample_bytree': 0.4,
#             'subsample': 0.3,
#             'min_child_weight': 10,
#             'lambda': 3,
#             'alpha': 2,
#             'gamma': 2,
# no burned

for random_split in [True, False]:
    print(random_split)
    sub = scores.loc[(scores.split == 'val') & (scores.random_split == random_split)]
    print(f'validation score = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')
    sub = scores.loc[(scores.split == 'test') & (scores.random_split == random_split)]
    print(f'testing score    = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')
    sub = scores.loc[(scores.split == 'train') & (scores.random_split == random_split)]
    print(f'training score   = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')

True
validation score = 0.6217459926747275
testing score    = 0.6954388168222952
training score   = 0.7912293935397181
False
validation score = 0.6256930787781588
testing score    = 0.6176102673640486
training score   = 0.7125557316541127


In [9]:
for validation_year in ['first', 'last']:
    print(validation_year)
    sub = scores.loc[(scores.split == 'val') & (scores.validation_year == validation_year)]
    print(f'validation score = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')
    sub = scores.loc[(scores.split == 'test') & (scores.validation_year == validation_year)]
    print(f'testing score    = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')
    sub = scores.loc[(scores.split == 'train') & (scores.validation_year == validation_year)]
    print(f'training score   = {(sub.r2 * sub.sample_size).sum() / sub.sample_size.sum()}')

first
validation score = 0.647022690965335
testing score    = 0.6325804760363014
training score   = 0.7452534745291781
last
validation score = 0.5898693726376628
testing score    = 0.6851369520529208
training score   = 0.7579545105806581


In [82]:
scores.loc[(scores.split == 'val') & (scores.random_split == False)]

,bias,mae,r2,model_name,split,realm,validation_year,random_split,sample_size,model_type
11,16.221206,37.730452,0.605638,xgb_afrotropic_last,val,afrotropic,last,False,1633454,xgb
14,16.280223,37.829253,0.603236,gb_afrotropic_last,val,afrotropic,last,False,1633454,gb
17,15.688924,39.327787,0.584920,rf_afrotropic_last,val,afrotropic,last,False,1633454,rf
29,1.024533,5.058862,0.793991,xgb_australia_last,val,australia,last,False,300193,xgb
32,1.440663,5.130379,0.785938,gb_australia_last,val,australia,last,False,300193,gb
35,1.562326,5.140293,0.790218,rf_australia_last,val,australia,last,False,300193,rf
47,0.654617,17.734039,0.601638,xgb_nearctic_last,val,nearctic,last,False,676925,xgb
50,0.692911,17.714689,0.603899,gb_nearctic_last,val,nearctic,last,False,676925,gb
53,0.873349,18.677573,0.581715,rf_nearctic_last,val,nearctic,last,False,676925,rf
65,21.428531,48.232903,0.579549,xgb_neotropic_last,val,neotropic,last,False,1581558,xgb


In [24]:
# weighted average performance 

scores['model_type'] = scores.model_name.apply(lambda x: x.split('_')[0])
for model, g in scores.loc[(scores.split == 'val') & (scores.random_split == False)].groupby('model_type'):
    print(model.ljust(3),  (g.r2 * g.sample_size).sum() / g.sample_size.sum())

gb  0.6070251900006757
rf  0.590250604711402
xgb 0.6086370353027087


In [80]:
# weighted average performance 

scores['model_type'] = scores.model_name.apply(lambda x: x.split('_')[0])
for model, g in scores.loc[(scores.split == 'val') & (scores.random_split == True)].groupby('model_type'):
    print(model.ljust(3),  (g.r2 * g.sample_size).sum() / g.sample_size.sum())

gb  0.6062574251978661
rf  0.5897037319571558
xgb 0.6073554210681192


In [79]:
scores.to_csv('default_rf_gb_scores.csv', index=False)

In [25]:
# simple average performance

scores.loc[(scores.split == 'val') & (scores.random_split == False)].groupby('model_type').r2.mean()

model_type
gb     0.642848
rf     0.629577
xgb    0.645204
Name: r2, dtype: float64

In [118]:
temporal_variability.to_csv('temporal_variability.csv', index=False)

In [112]:
temporal_variability = pd.read_csv('temporal_variability.csv')

In [113]:
temporal_variability['realm'] = temporal_variability.model_name.apply(lambda x: x.split('_')[1])
temporal_variability['model_type'] = temporal_variability.model_name.apply(lambda x: x.split('_')[0])

sample_size = scores.loc[
    (scores.random_split == True) & (scores.model_name.str.startswith('xgb'))
].groupby('realm').sample_size.sum()

In [120]:
# weighted average 
merged = temporal_variability.loc[temporal_variability.random_split != True].merge(sample_size, how='left', on='realm')
name_dict = {
    'gb': 'gradient boosting',
    'ground': 'lidar derived',
    'rf': 'random forest',
    'xgb': 'xgboost'
}
merged['model_type'] = merged.model_type.apply(lambda x: name_dict[x])

print('Biomass MAE between years 2007 and 2008 of the same location using different model architecture')
print('')
for model, g in merged.groupby('model_type'):
    print(model.ljust(20),  np.round((g.mae * g.sample_size).sum() / g.sample_size.sum(), 4))

Biomass MAE between years 2007 and 2008 of the same location using different model architecture

gradient boosting    17.7812
lidar derived        33.0125
random forest        15.9057
xgboost              17.9184


In [55]:
# simple average 
temporal_variability.loc[temporal_variability.random_split != True].merge(sample_size, how='left', on='realm').groupby('model_type').mae.mean()

model_type
gb        14.136758
ground    25.892038
rf        12.603001
xgb       14.137349
Name: mae, dtype: float64

In [ ]:
print('done')

In [7]:

#             'learning_rate': 0.05,
#             'max_depth': 10,
#             'colsample_bytree': 0.7,
#             'subsample': 0.7,
#             'min_child_weight': 4,
scores.loc[scores.split == 'val'].r2.mean()

0.623497651124284

In [16]:
print('done')

done


In [20]:
df = []
for realm in realms:
    for strategy in ['first', 'last']:
        df.append(pd.read_csv(f'{realm}_{strategy}.csv'))
        
df = pd.concat(df)

In [81]:
scores = pd.read_csv('HPO_1.csv')

In [26]:
df.loc[df.split == 'test'].groupby(['learning_rate', 'max_depth', 'colsample_bytree', 'lambda']).mean().sort_values(by='r2')

Unnamed: 0      bias  \
learning_rate max_depth colsample_bytree lambda                         
0.10          8         0.5              1.5            4.0  2.308906   
              10        0.5              2.0           34.0  2.349971   
                                         1.0           28.0  2.270306   
              8         0.5              1.0            1.0  2.353026   
                                         2.0            7.0  2.322086   
              10        0.5              1.5           31.0  2.306159   
0.05          8         0.5              1.0           55.0  2.205334   
                                         1.5           58.0  2.233750   
0.10          8         0.7              1.0           10.0  2.221195   
                                         1.5           13.0  2.148057   
0.05          8         0.5              2.0           61.0  2.247323   
0.10          10        0.7              1.0           37.0  1.863723   
              8         0.7              2.0           16.0  2.291688   
                        0.9              1.0           19.0  2.153713   
0.05          10        0.5              1.5           85.0  2.248638   
0.10          8         0.9              2.0           25.0  2.135202   
0.05          10        0.5              1.0           82.0  2.194941   
0.10          10        0.7              1.5           40.0  2.196090   
                        0.9              1.0           46.0  2.062420   
                        0.7              2.0           43.0  1.976320   
              8         0.9              1.5           22.0  2.273610   
0.05          10        0.5              2.0           88.0  2.161444   
0.10          10        0.9              2.0           52.0  1.875923   
0.05          8         0.7              1.5           67.0  2.172436   
0.10          10        0.9              1.5           49.0  2.143239   
0.05          8         0.9              1.5           76.0  2.205283   
                        0.7              2.0           70.0  2.245443   
                                         1.0           64.0  2.214691   
              10        0.7              1.5           94.0  2.161761   
              8         0.9              1.0           73.0  2.226369   
                                         2.0           79.0  2.205607   
              10        0.9              1.5          103.0  2.126695   
                        0.7              1.0           91.0  2.174415   
                                         2.0           97.0  2.201969   
                        0.9              1.0          100.0  2.187778   
                                         2.0          106.0  2.146186   

                                                       mae        r2  \
learning_rate max_depth colsample_bytree lambda                        
0.10          8         0.5              1.5     26.397830  0.681160   
              10        0.5              2.0     26.181475  0.681359   
                                         1.0     26.203374  0.681397   
              8         0.5              1.0     26.385209  0.681407   
                                         2.0     26.377354  0.681430   
              10        0.5              1.5     26.179458  0.681647   
0.05          8         0.5              1.0     26.242036  0.683585   
                                         1.5     26.236306  0.683650   
0.10          8         0.7              1.0     26.024090  0.683702   
                                         1.5     26.027765  0.683956   
0.05          8         0.5              2.0     26.252736  0.684218   
0.10          10        0.7              1.0     25.914860  0.684224   
              8         0.7              2.0     25.984036  0.684821   
                        0.9              1.0     25.829956  0.685134   
0.05          10        0.5              1.5     26.031687  0.685307   
0.10          8         0.9              2.0     2

In [21]:
df_train, df_test, df_val = m.train_test_split_based_on_year(df, val_strategy='last', random_train_test=False)
print(f'    training sample size = {len(df_train)}')
print(f'    testing sample size = {len(df_test)}')
print(f'    eval sample size = {len(df_val)}')

    training sample size = 1876208
    testing sample size = 359029
    eval sample size = 324938


In [39]:
scores = []
model_class = m.xgb_model
    model = model_class(
        realm=realm, 
        df_train=df_train,
        df_test=df_test,
        output_folder='s3://carbonplan-climatetrace/v1/models/',
        params=params,
        overwrite=True
    )
        
    for split, df in zip(('train', 'test', 'val'), (df_train, df_test, df_val)):
        model_score = model.evaluate(df)
        model_score['model_name'] = model.name
        model_score['split'] = split
        model_score.update(params)
        scores.append(model_score)

{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'n_estimators': 1000, 'random_state': 42, 'learning_rate': 0.1, 'max_depth': 10}
    Building xgb_nearctic model
[0]	validation_0-rmse:92.50160
[1]	validation_0-rmse:87.49003
[2]	validation_0-rmse:83.16765
[3]	validation_0-rmse:79.42314
[4]	validation_0-rmse:76.22594
[5]	validation_0-rmse:73.46867
[6]	validation_0-rmse:71.06100
[7]	validation_0-rmse:69.00629
[8]	validation_0-rmse:67.22349
[9]	validation_0-rmse:65.71007
[10]	validation_0-rmse:64.45616
[11]	validation_0-rmse:63.38517
[12]	validation_0-rmse:62.46336
[13]	validation_0-rmse:61.69775
[14]	validation_0-rmse:61.06208
[15]	validation_0-rmse:60.48203
[16]	validation_0-rmse:59.99840
[17]	validation_0-rmse:59.63692
[18]	validation_0-rmse:59.27531
[19]	validation_0-rmse:58.96634
[20]	validation_0-rmse:58.67696
[21]	validation_0-rmse:58.46461
[22]	validation_0-rmse:58.26601
[23]	validation_0-rmse:58.09242
[24]	validation_0-rmse:57.89921
[25]	validation_0-rmse:57.83263
[26]	val

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'n_estimators': 2000, 'random_state': 42, 'learning_rate': 0.1, 'max_depth': 10}
    Building xgb_nearctic model
[0]	validation_0-rmse:92.50160
[1]	validation_0-rmse:87.49004
[2]	validation_0-rmse:83.16765
[3]	validation_0-rmse:79.42314
[4]	validation_0-rmse:76.22594
[5]	validation_0-rmse:73.46867
[6]	validation_0-rmse:71.06100
[7]	validation_0-rmse:69.00628
[8]	validation_0-rmse:67.22348
[9]	validation_0-rmse:65.71008
[10]	validation_0-rmse:64.45616
[11]	validation_0-rmse:63.38517
[12]	validation_0-rmse:62.46336
[13]	validation_0-rmse:61.69776
[14]	validation_0-rmse:61.06208
[15]	validation_0-rmse:60.48203
[16]	validation_0-rmse:59.99840
[17]	validation_0-rmse:59.63692
[18]	validation_0-rmse:59.27531
[19]	validation_0-rmse:58.96634
[20]	validation_0-rmse:58.67696
[21]	validation_0-rmse:58.46461
[22]	validation_0-rmse:58.26601
[23]	validation_0-rmse:58.09242
[24]	validation_0-rmse:57.89922
[25]	validation_0-rmse:57.83263
[26]	val

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'n_estimators': 3000, 'random_state': 42, 'learning_rate': 0.1, 'max_depth': 10}
    Building xgb_nearctic model
[0]	validation_0-rmse:92.50160
[1]	validation_0-rmse:87.49004
[2]	validation_0-rmse:83.16765
[3]	validation_0-rmse:79.42314
[4]	validation_0-rmse:76.22594
[5]	validation_0-rmse:73.46866
[6]	validation_0-rmse:71.06100
[7]	validation_0-rmse:69.00628
[8]	validation_0-rmse:67.22348
[9]	validation_0-rmse:65.71007
[10]	validation_0-rmse:64.45616
[11]	validation_0-rmse:63.38517
[12]	validation_0-rmse:62.46336
[13]	validation_0-rmse:61.69776
[14]	validation_0-rmse:61.06208
[15]	validation_0-rmse:60.48203
[16]	validation_0-rmse:59.99840
[17]	validation_0-rmse:59.63691
[18]	validation_0-rmse:59.27531
[19]	validation_0-rmse:58.96634
[20]	validation_0-rmse:58.67696
[21]	validation_0-rmse:58.46461
[22]	validation_0-rmse:58.26601
[23]	validation_0-rmse:58.09242
[24]	validation_0-rmse:57.89922
[25]	validation_0-rmse:57.83263
[26]	val

...

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'n_estimators': 2000, 'random_state': 42, 'learning_rate': 0.02, 'max_depth': 14}
    Building xgb_nearctic model
[0]	validation_0-rmse:97.08012
[1]	validation_0-rmse:95.90262
[2]	validation_0-rmse:94.75969
[3]	validation_0-rmse:93.64358
[4]	validation_0-rmse:92.56133
[5]	validation_0-rmse:91.49529
[6]	validation_0-rmse:90.47224
[7]	validation_0-rmse:89.47387
[8]	validation_0-rmse:88.49540
[9]	validation_0-rmse:87.54831
[10]	validation_0-rmse:86.62385
[11]	validation_0-rmse:85.72732
[12]	validation_0-rmse:84.85838
[13]	validation_0-rmse:84.00948
[14]	validation_0-rmse:83.19078
[15]	validation_0-rmse:82.38027
[16]	validation_0-rmse:81.60738
[17]	validation_0-rmse:80.85918
[18]	validation_0-rmse:80.11949
[19]	validation_0-rmse:79.41093
[20]	validation_0-rmse:78.71761
[21]	validation_0-rmse:78.04800
[22]	validation_0-rmse:77.37925
[23]	validation_0-rmse:76.75506
[24]	validation_0-rmse:76.14045
[25]	validation_0-rmse:75.54469
[26]	va

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'objective': 'reg:squarederror', 'eval_metric': 'rmse', 'n_estimators': 3000, 'random_state': 42, 'learning_rate': 0.02, 'max_depth': 14}
    Building xgb_nearctic model
[0]	validation_0-rmse:97.08012
[1]	validation_0-rmse:95.90262
[2]	validation_0-rmse:94.75969
[3]	validation_0-rmse:93.64357
[4]	validation_0-rmse:92.56133
[5]	validation_0-rmse:91.49530
[6]	validation_0-rmse:90.47224
[7]	validation_0-rmse:89.47387
[8]	validation_0-rmse:88.49540
[9]	validation_0-rmse:87.54832
[10]	validation_0-rmse:86.62386
[11]	validation_0-rmse:85.72732
[12]	validation_0-rmse:84.85838
[13]	validation_0-rmse:84.00948
[14]	validation_0-rmse:83.19078
[15]	validation_0-rmse:82.38028
[16]	validation_0-rmse:81.60738
[17]	validation_0-rmse:80.85918
[18]	validation_0-rmse:80.11949
[19]	validation_0-rmse:79.41093
[20]	validation_0-rmse:78.71761
[21]	validation_0-rmse:78.04800
[22]	validation_0-rmse:77.37925
[23]	validation_0-rmse:76.75506
[24]	validation_0-rmse:76.14045
[25]	validation_0-rmse:75.54469
[26]	va

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [40]:
r = pd.DataFrame(scores)
r.loc[r.split == 'val'].sort_values(by='r2')

,bias,mae,r2,model_name,split,learning_rate,max_depth,n_estimators
2,-0.852048,23.668822,0.507676,xgb_nearctic,val,0.10,10,1000
5,-0.852048,23.668822,0.507676,xgb_nearctic,val,0.10,10,2000
8,-0.852048,23.668822,0.507676,xgb_nearctic,val,0.10,10,3000
11,-0.934256,23.265791,0.508721,xgb_nearctic,val,0.10,14,1000
14,-0.934256,23.265791,0.508721,xgb_nearctic,val,0.10,14,2000
17,-0.934256,23.265791,0.508721,xgb_nearctic,val,0.10,14,3000
35,-0.947551,23.138256,0.512086,xgb_nearctic,val,0.05,14,3000
32,-0.947551,23.138256,0.512086,xgb_nearctic,val,0.05,14,2000
29,-0.947551,23.138256,0.512086,xgb_nearctic,val,0.05,14,1000
26,-0.804719,23.598356,0.512300,xgb_nearctic,val,0.05,10,3000


In [41]:
r = pd.DataFrame(scores)
r.loc[r.split == 'test'].sort_values(by='r2')

,bias,mae,r2,model_name,split,learning_rate,max_depth,n_estimators
1,-2.561562,22.603937,0.529733,xgb_nearctic,test,0.10,10,1000
4,-2.561562,22.603937,0.529733,xgb_nearctic,test,0.10,10,2000
7,-2.561562,22.603937,0.529733,xgb_nearctic,test,0.10,10,3000
10,-2.747026,22.235418,0.532029,xgb_nearctic,test,0.10,14,1000
13,-2.747026,22.235418,0.532029,xgb_nearctic,test,0.10,14,2000
16,-2.747026,22.235418,0.532029,xgb_nearctic,test,0.10,14,3000
28,-2.792690,22.112319,0.534002,xgb_nearctic,test,0.05,14,1000
31,-2.792690,22.112319,0.534002,xgb_nearctic,test,0.05,14,2000
34,-2.792690,22.112319,0.534002,xgb_nearctic,test,0.05,14,3000
43,-2.527872,22.509412,0.536906,xgb_nearctic,test,0.02,10,3000


In [36]:
r = pd.DataFrame(scores)
r.loc[r.split == 'val'].sort_values(by='r2')

,bias,mae,r2,model_name,split,learning_rate,max_depth
2,-1.037318,24.771967,0.484516,xgb_nearctic,val,0.200,6
29,-0.995037,24.560283,0.493952,xgb_nearctic,val,0.075,6
38,-0.927733,24.503957,0.494016,xgb_nearctic,val,0.050,6
11,-0.858568,24.668030,0.494875,xgb_nearctic,val,0.150,6
8,-1.259345,23.443574,0.495102,xgb_nearctic,val,0.200,14
5,-0.980468,23.853668,0.497390,xgb_nearctic,val,0.200,10
20,-0.828704,24.492755,0.501537,xgb_nearctic,val,0.100,6
14,-0.814873,23.743686,0.504668,xgb_nearctic,val,0.150,10
17,-0.983394,23.333829,0.505280,xgb_nearctic,val,0.150,14
32,-1.125266,23.683123,0.506540,xgb_nearctic,val,0.075,10


In [9]:
#     'learning_rate': 0.05,
#     'max_depth': 10,
#     'colsample_bytree': 0.6,
#     'subsample': 0.6,
#     'min_child_weight': 4,
# random split in train/test
scores

,bias,mae,r2,model_name,split,realm
0,-1.713595e-12,51.629270,-2.220446e-16,baseline_nearctic,train,nearctic
1,-4.116822e-04,51.626876,-2.666312e-11,baseline_nearctic,test,nearctic
2,-1.035291e+00,50.380847,-1.869265e-04,baseline_nearctic,val,nearctic
3,-5.482044e-03,16.542344,8.307906e-01,xgb_nearctic,train,nearctic
4,-1.688718e-02,19.162086,6.639486e-01,xgb_nearctic,test,nearctic
5,1.095924e+00,20.959087,5.445431e-01,xgb_nearctic,val,nearctic


In [44]:
#     'learning_rate': 0.05,
#     'max_depth': 10,
#     'colsample_bytree': 0.6,
#     'subsample': 0.6,
#     'min_child_weight': 4,
# year split in train test 
xgb.evaluate(df_val)

/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/core.py:101: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(
/srv/conda/envs/notebook/lib/python3.8/site-packages/xgboost/data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


{'bias': 0.8843416267143598,
 'mae': 22.294730590031673,
 'r2': 0.5440653435394311}

In [15]:
# from sklearn.preprocessing import OneHotEncoder
# igbp_encoder = OneHotEncoder(sparse=False, categories='auto', handle_unknown='ignore').fit(df_train[['igbp']])
#     # one hot encoding for igbp 
#     encoded_igbp = igbp_encoder.transform(X[['igbp']])
#     X = X.drop(['igbp'], axis=1)
#     for i in range(encoded_igbp.shape[1]):
#         X[f'igbp_cat_{str(i+1)}'] = encoded_igbp[:, i]
